In [ ]:
import numpy as np
import qml
import sys
sys.path.insert(0, '/home/misa/APDFT/prototyping/atomic_energies/')
import qml_interface as qmi

import matplotlib
matplotlib.use("Qt5Agg")
from matplotlib import pyplot as plt

# Indices correct?

compare indices for manually created small atomic kernel with full kernel

In [ ]:
data, molecule_size = qmi.load_alchemy_data(qmi.wrapper_alch_data())

In [ ]:
# choose random molecules
global_tr, global_test = qmi.get_indices(len(molecule_size), 3)
global_tr = np.array([ 14, 519, 549])
tr_ind = qmi.get_local_idx(global_tr, molecule_size)
test_ind = qmi.get_local_idx(global_test, molecule_size)

In [ ]:
# calculate representation and atomic kernels
reps = []
for i in global_tr:
    rep_fchl = qml.fchl.generate_representation(data[i][:, 1:4], data[i][:,0], max_size=molecule_size[i])
    reps.append(rep_fchl)

In [ ]:
sigma = 432.8761281083057
kernel00 = qml.fchl.fchl_scalar_kernels.get_atomic_kernels(reps[0], reps[0], kernel_args={"sigma":[sigma],})[0]
kernel01 = qml.fchl.fchl_scalar_kernels.get_atomic_kernels(reps[0], reps[1], kernel_args={"sigma":[sigma],})[0]
kernel02 = qml.fchl.fchl_scalar_kernels.get_atomic_kernels(reps[0], reps[2], kernel_args={"sigma":[sigma],})[0]
kernel11 = qml.fchl.fchl_scalar_kernels.get_atomic_kernels(reps[1], reps[1], kernel_args={"sigma":[sigma],})[0]
kernel12 = qml.fchl.fchl_scalar_kernels.get_atomic_kernels(reps[1], reps[2], kernel_args={"sigma":[sigma],})[0]
kernel22 = qml.fchl.fchl_scalar_kernels.get_atomic_kernels(reps[2], reps[2], kernel_args={"sigma":[sigma],})[0]

In [ ]:
kernel3x3 = np.block([[kernel00, kernel01, kernel02], [kernel01.T, kernel11, kernel12], [kernel02.T, kernel12.T, kernel22]])

In [ ]:
# kernel between molecule at index 0 and molecule at index 2

# select correct indices
start_idx = molecule_size[14] + molecule_size[519]
print(start_idx)
end_idx = start_idx + molecule_size[549] - 1
print(end_idx)

# pick slice
kernel3x3_02 = kernel3x3[0:16, 26:39]

In [ ]:
# load kernel

In [ ]:
kernel_full = np.loadtxt('/home/misa/APDFT/prototyping/atomic_energies/results/analyse_learning/FCHL/full_kernel_sig432.8761281083057')

In [ ]:
# indices in full kernel
start_idx_full_row = molecule_size[0:14].sum()
end_idx_full_row = start_idx_full_row + molecule_size[14]-1

In [ ]:
start_idx_full_column = molecule_size[0:549].sum()
end_idx_full_column= start_idx_full_column + molecule_size[549]-1

In [ ]:
# select correct indices
kernel_full_slice = kernel_full[start_idx_full_row:end_idx_full_row+1, start_idx_full_column:end_idx_full_column+1]

In [ ]:
kernel02 - kernel_full_slice

In [ ]:
kernel3x3_02 - kernel_full_slice

## indices are correctly assigned in full matrix
## what about kernel splitting during hyperparameter splitting?

In [ ]:
print(start_idx_full_row)
print(end_idx_full_row)

In [ ]:
# split kernel
tr_kernel, test_kernel = qmi.split_kernel(kernel_full, tr_ind, test_ind)

In [ ]:
# correct kernel selected? yes identical to manually computed kernel
print(np.amax(tr_kernel-kernel3x3))
print(np.amin(tr_kernel-kernel3x3))

## Difference between kernels with different sigma

In [ ]:
import numpy as np
import qml
import sys
sys.path.insert(0, '/home/misa/APDFT/prototyping/atomic_energies/')
import qml_interface as qmi

In [ ]:
kernel_full1 = np.loadtxt('/home/misa/APDFT/prototyping/atomic_energies/results/analyse_learning/FCHL/full_kernel_sig432.8761281083057')

In [ ]:
kernel_full2 = np.loadtxt('/home/misa/APDFT/prototyping/atomic_energies/results/analyse_learning/FCHL/full_kernel_sig10000.0')

In [ ]:
kernel_full2-kernel_full1

## Training error as function of sigma

In [ ]:
import numpy as np
import qml
import sys
sys.path.insert(0, '/home/misa/APDFT/prototyping/atomic_energies/')
import qml_interface as qmi
import glob
import re

# paths to kernels
paths = glob.glob('/home/misa/APDFT/prototyping/atomic_energies/results/analyse_learning/FCHL/full_kernel_sig*')

# indices for training kernel selection
global_tr = np.array([ 14, 519, 549])
data, molecule_size = qmi.load_alchemy_data(qmi.wrapper_alch_data())
tr_ind = qmi.get_local_idx(global_tr, molecule_size)

global_tr, global_test = qmi.get_indices(len(molecule_size), 3)
global_tr = np.array([ 14, 519, 549])
tr_ind = qmi.get_local_idx(global_tr, molecule_size)
test_ind = qmi.get_local_idx(global_test, molecule_size)

error = []
kernel_diff= []

for p in paths:
    if re.search('lam',p):
        pass
    else:
        kernel_full1 = np.loadtxt(p)
        
        # calculate training error
        tr_kernel, test_kernel = qmi.split_kernel(kernel_full1, tr_ind, test_ind)
        labels = qmi.generate_label_vector(data, molecule_size.sum(), value='atomisation')
        tr_label = labels[tr_ind]

        coeffs = qml.math.cho_solve(tr_kernel, tr_label)
        pred_label = np.dot(tr_kernel, coeffs)
        tr_error = np.abs(tr_label-pred_label)
        
        error.append([float(p.split('sig')[1]), tr_error.mean()])
        
        # maximum difference between to elements in training kernel
        max_diff = np.amax(tr_kernel) - np.amin(tr_kernel)
        kernel_diff.append([float(p.split('sig')[1]), max_diff])

In [ ]:
error.sort()
error
tr_errors = np.array(error)

In [ ]:
kernel_diff.sort()
kernel_diff_np = np.array(kernel_diff)
kernel_diff_np

In [ ]:
# tr_errors = np.array([[0.1, 9.533845087234126e-07],
#  [0.2848035868435802, 1.1146930695175114e-05],
#  [0.8111308307896871, 0.0002756277498121068],
#  [2.3101297000831593, 0.0035897837548015087],
#  [6.579332246575679, 3273337966246805.5],
#  [18.73817422860383, 150713133386679.62],
#  [53.36699231206307, 4.2190757635430615e+18],
#  [151.99110829529332, 1.9391011745425462e+21],
#  [432.8761281083057, 1.3728786113167146e+22],
#  [1232.8467394420659, 5.937093237082583e+22],
#  [3511.1917342151273, 1.8727704323267042e+27],
#  [10000.0, 3.2333931384220235e+23]])

In [ ]:
plt.plot(tr_errors[:,0], tr_errors[:,1], '-o')
plt.xscale('log')
plt.yscale('log')
plt.xlabel(r'$\sigma$')
plt.ylabel('training error')

In [ ]:
plt.plot(kernel_diff_np[:,0], kernel_diff_np[:,1], '-o')
plt.xscale('log')
plt.yscale('log')
plt.xlabel(r'$\sigma$')
plt.ylabel('Max Kerneldiff')

## Delta learning 
according to Anders changes in the Hartree range can be problematic for FCHL -> subtract average atomic energy for every element


In [31]:
import numpy as np
import qml
import sys
sys.path.insert(0, '/home/misa/APDFT/prototyping/atomic_energies/')
import qml_interface as qmi
import glob
import re

import matplotlib
matplotlib.use("Qt5Agg")
from matplotlib import pyplot as plt

In [ ]:
# paths to kernels
paths = glob.glob('/home/misa/APDFT/prototyping/atomic_energies/results/analyse_learning/FCHL/full_kernel_sig*')

# indices for training kernel selection
global_tr = np.array([ 14, 519, 549])
data, molecule_size = qmi.load_alchemy_data(qmi.wrapper_alch_data())
tr_ind = qmi.get_local_idx(global_tr, molecule_size)

global_tr, global_test = qmi.get_indices(len(molecule_size), 3)
# global_tr = np.array([ 14, 519, 549])
tr_ind = qmi.get_local_idx(global_tr, molecule_size)
test_ind = qmi.get_local_idx(global_test, molecule_size)

error = []
kernel_diff= []

delta = False

for p in paths:
    if re.search('lam',p):
        pass
    else:
        print("Workin on kernel {}".format(p))
        kernel_full1 = np.loadtxt(p)
        
        # calculate training error
        tr_kernel, test_kernel = qmi.split_kernel(kernel_full1, tr_ind, test_ind)
        labels = qmi.generate_label_vector(data, molecule_size.sum(), value='alch_pot')
        if delta:
            tr_label = labels[tr_ind] - qmi.get_label_delta('alch_pot', tr_ind, data, molecule_size)
        else:
            tr_label = labels[tr_ind]

        coeffs = qml.math.cho_solve(tr_kernel, tr_label)
        pred_label = np.dot(tr_kernel, coeffs)
        tr_error = np.abs(tr_label-pred_label)
        
        error.append([float(p.split('sig')[1]), tr_error.mean()])
        
        # maximum difference between to elements in training kernel
        max_diff = np.amax(tr_kernel) - np.amin(tr_kernel)
        kernel_diff.append([float(p.split('sig')[1]), max_diff])

Workin on kernel /home/misa/APDFT/prototyping/atomic_energies/results/analyse_learning/FCHL/full_kernel_sig3511.1917342151273
Workin on kernel /home/misa/APDFT/prototyping/atomic_energies/results/analyse_learning/FCHL/full_kernel_sig0.8111308307896871
Workin on kernel /home/misa/APDFT/prototyping/atomic_energies/results/analyse_learning/FCHL/full_kernel_sig10000.0
Workin on kernel /home/misa/APDFT/prototyping/atomic_energies/results/analyse_learning/FCHL/full_kernel_sig151.99110829529332
Workin on kernel /home/misa/APDFT/prototyping/atomic_energies/results/analyse_learning/FCHL/full_kernel_sig53.36699231206307


In [ ]:
bae = '/home/misa/APDFT/prototyping/atomic_energies/results/analyse_learning/FCHL/delta_learning/atomisation/'

In [12]:
# training error
error.sort()
if delta:
    np.savetxt(bae+'delta_tr_error_alch_pot_3mol', error, delimiter='\t', header='sigma \t MAE (Ha)')
else:
    np.savetxt(bae+'tr_error_alch_pot_3mol', error, delimiter='\t', header='sigma \t MAE (Ha)')

In [13]:
# maximum difference between kernel elements
kernel_diff.sort()
if delta:
    np.savetxt(bae+'delta_max_kernel_diff_alch_pot_3mol', kernel_diff, delimiter='\t', header='sigma \t Max Kernel diff (Ha)')
else:
    np.savetxt(bae+'max_kernel_diff_alch_pot_3mol', kernel_diff, delimiter='\t', header='sigma \t Max Kernel diff (Ha)')


In [28]:
# training labels
if delta:
    labels = qmi.generate_label_vector(data, molecule_size.sum(), value='alch_pot')
    tr_label = labels[tr_ind] - qmi.get_label_delta('alch_pot', tr_ind, data, molecule_size)
    np.savetxt(bae+'delta_tr_label_alch_pot_3mol', tr_label, delimiter='\t', header='sigma \t tr _labels (Ha)')
else:
    labels = qmi.generate_label_vector(data, molecule_size.sum(), value='alch_pot')
    tr_label = labels[tr_ind]
    np.savetxt(bae+'tr_label_alch_pot_3mol', tr_label, delimiter='\t', header='sigma \t tr _labels (Ha)')

In [33]:
fig, ax = plt.subplots(1,3)

glob.glob('/home/misa/APDFT/prototyping/atomic_energies/results/analyse_learning/FCHL/delta_learning/*')

['/home/misa/APDFT/prototyping/atomic_energies/results/analyse_learning/FCHL/delta_learning/delta_tr_label_alch_pot_3mol',
 '/home/misa/APDFT/prototyping/atomic_energies/results/analyse_learning/FCHL/delta_learning/tr_label_alch_pot_3mol',
 '/home/misa/APDFT/prototyping/atomic_energies/results/analyse_learning/FCHL/delta_learning/delta_max_kernel_diff_alch_pot_3mol',
 '/home/misa/APDFT/prototyping/atomic_energies/results/analyse_learning/FCHL/delta_learning/tr_error_alch_pot_3mol',
 '/home/misa/APDFT/prototyping/atomic_energies/results/analyse_learning/FCHL/delta_learning/max_kernel_diff_alch_pot_3mol',
 '/home/misa/APDFT/prototyping/atomic_energies/results/analyse_learning/FCHL/delta_learning/delta_tr_error_alch_pot_3mol']

In [51]:
paths = glob.glob('/home/misa/APDFT/prototyping/atomic_energies/results/analyse_learning/FCHL/delta_learning/atomisation/*')
paths.sort()
fig, ax = plt.subplots(1,3)
fig.suptitle('Training of Alchemical potential delta learning vs normal')
ax[0].set_title('Training error')
ax[1].set_title('Max kernel difference')
ax[2].set_title('Training label distribution')

for idx, p in enumerate(paths):
    data = np.loadtxt(p)
    
    if idx == 0:
        ax[1].plot(data[:,0], data[:,1], '-o', label = 'delta')
        
    elif idx == 1:
        ax[0].plot(data[:,0], data[:,1], '-o', label = 'delta')
        
    elif idx == 2:
        ax[2].plot(data, '-o', label = 'delta')
        
    elif idx == 3:
        ax[1].plot(data[:,0], data[:,1], '-o', label = 'raw')
        
    elif idx == 4:
        ax[0].plot(data[:,0], data[:,1], '-o', label = 'raw')
        
    elif idx == 5:
        ax[2].plot(data, '-o', label = 'raw')
        
for i in range(2):
    ax[i].set_xscale('log')
    ax[i].set_yscale('log')
    
for i in range(3):
    ax[i].legend()

In [42]:
data

array([ 0.43790982,  0.31174752, -0.19087957,  0.00055941, -0.08071221,
        0.35157007,  0.31288996, -0.18429247,  0.08468616,  0.07446131,
       -0.07385931,  0.36240425,  0.21908644,  0.32092084, -0.37918468,
        0.08820152, -0.18564799,  0.18366308, -0.29050073, -0.33317767,
       -0.06473998, -0.08834251, -0.03567742, -0.05682352,  0.03759281,
       -0.14970072,  0.18841768,  0.0427309 , -0.10148092,  0.04299451,
       -0.34494079, -0.30081433, -0.07426643, -0.10954644,  0.09353424,
        0.11902352, -0.01218055,  0.02400782, -0.1427479 , -0.13928028,
        0.04239454])